In [ ]:
!pip install timm -q
!pip install fastai --upgrade --q
!pip install fastaudio --q
exit()

This will restart the notebook, just continiue the running the codes after 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%time
!unzip -o -q '/content/drive/MyDrive/swahili-audio-classification/Swahili_words.zip' -d "/content/Swahili_words"

CPU times: user 70 ms, sys: 16.2 ms, total: 86.2 ms
Wall time: 11.9 s


In [ ]:
import warnings
from fastai.vision.all import *
from fastaudio.core.all import *
from fastaudio.augment.all import *
import numpy as np

path = "/content/drive/MyDrive/swahili-audio-classification/"
warnings.filterwarnings('ignore')
seed = 2020

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:47: UserWarning: "torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE" flag is deprecated and will be removed in 0.9.0. Please remove the use of flag.
  '"torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE" flag is deprecated and will be removed in 0.9.0. '


In [ ]:
train = pd.read_csv(path+"Train.csv")
test = pd.read_csv(path+"Test.csv")
ss = pd.read_csv(path+"SampleSubmission.csv")

train

,Word_id,Swahili_word,English_translation
0,id_v8rz06e6rv31.wav,mbili,two
1,id_vmbwicdpfn68.wav,tatu,three
2,id_injlouhxg1hg.wav,ndio,yes
3,id_gdq23p6xgzya.wav,nne,four
4,id_lpstklz51zpz.wav,nane,eight
...,...,...,...
4195,id_ho8aqa3s42zp.wav,hapana,no
4196,id_3lhkgf9dqr4c.wav,kumi,ten
4197,id_ixb7vv4i0zct.wav,tano,five
4198,id_7tboov3kavg4.wav,nne,four


In [ ]:
from sklearn.model_selection import StratifiedKFold

train["Word_id"] = "/content/Swahili_words/" + train["Word_id"]
test["Word_id"] = "/content/Swahili_words/" + test["Word_id"]

skf = StratifiedKFold(10, random_state=seed, shuffle=True)
for fold, (_, val_inx) in enumerate(skf.split(train, train.Swahili_word)):
    train.loc[val_inx, 'fold'] = fold
train.fold = train.fold.astype(int)
train

,Word_id,Swahili_word,English_translation,fold
0,/content/Swahili_words/id_v8rz06e6rv31.wav,mbili,two,0
1,/content/Swahili_words/id_vmbwicdpfn68.wav,tatu,three,8
2,/content/Swahili_words/id_injlouhxg1hg.wav,ndio,yes,4
3,/content/Swahili_words/id_gdq23p6xgzya.wav,nne,four,6
4,/content/Swahili_words/id_lpstklz51zpz.wav,nane,eight,0
...,...,...,...,...
4195,/content/Swahili_words/id_ho8aqa3s42zp.wav,hapana,no,4
4196,/content/Swahili_words/id_3lhkgf9dqr4c.wav,kumi,ten,8
4197,/content/Swahili_words/id_ixb7vv4i0zct.wav,tano,five,3
4198,/content/Swahili_words/id_7tboov3kavg4.wav,nne,four,3


In [ ]:
def random_seed(seed_value, use_cuda): 
   np.random.seed(seed_value)
   torch.manual_seed(seed_value) 

   random.seed(seed_value)

   if use_cuda: 
      torch.cuda.manual_seed(seed_value) 
      torch.cuda.manual_seed_all(seed_value) 

      torch.backends.cudnn.deterministic = True 

      torch.backends.cudnn.benchmark = False 

random_seed(seed, True)
set_seed(seed, True)

In [ ]:
augs = [
        AddNoiseGPU, ChangeVolumeGPU, DeltaGPU, MaskFreqGPU, MaskTimeGPU, SignalCutoutGPU, SignalLossGPU, TfmResizeGPU
]

In [ ]:
params = {'n_fft':2048, 'hop_length':308, 'n_mels':224, 'sample_rate':16000, 'win_length':1024}
cfg = AudioConfig.BasicMelSpectrogram(**params) # pull up the voice config
aud2spec = AudioToSpec.from_cfg(cfg) # let audio to spec use the config

In [ ]:
class AudioNormalize(Transform):
    "Normalizes a single `AudioTensor`."
    def encodes(self, x:AudioTensor): return (x-x.mean()) / x.std()

def get_dls(fold):
    torch.cuda.empty_cache()
    audio = DataBlock(blocks=(AudioBlock(force_mono=True, crop_signal_to=5000), 
                            CategoryBlock),
                        splitter=MaskSplitter(train.fold == fold),
                        get_x = ColReader(0),
                        get_y=ColReader(1),
                        item_tfms = [AudioNormalize],
                        batch_tfms = [aud2spec, ChangeVolume, SignalLoss]        
        )
    dls = audio.dataloaders(train, bs=64, num_workers=0, device='cuda')
    dls.rng.seed(seed)

    return dls

In [ ]:
predictions = []
for fold in [4]:
    print(f"\n\nFold {fold}")
    dls = get_dls(fold)
    model_name = f'/content/fold{fold}-model'
    learn = cnn_learner(dls, resnet50, CrossEntropyLossFlat(), metrics=accuracy, n_in=1)
    callbacks = [
             SaveModelCallback(monitor='valid_loss', fname=model_name) 
             ]
    learn.unfreeze()
    lr = learn.lr_find(show_plot=False)[0]
    learn.fit_one_cycle(16, wd=0.1, lr_max=lr, cbs=callbacks)

    tdl = learn.dls.test_dl(test["Word_id"]) # quicly create test data loader

    model = learn.load(model_name)
    test_preds_tta, test_labels_tta = model.tta(dl=tdl, n=32)
    predictions.append(test_preds_tta)



Fold 3


epoch,train_loss,valid_loss,accuracy,time
0,1.828571,2.696691,0.507143,01:14
1,0.906021,1.149457,0.728571,01:14
2,0.776587,7.554482,0.211905,01:14
3,0.731812,17.736679,0.140476,01:14
4,0.504422,0.692675,0.821429,01:14
5,0.378830,0.601470,0.850000,01:14
6,0.368061,2.032750,0.697619,01:14
7,0.331645,11.706565,0.226190,01:13
8,0.303838,0.873595,0.776190,01:14
9,0.227545,0.529413,0.850000,01:14


Better model found at epoch 0 with valid_loss value: 2.6966910362243652.
Better model found at epoch 1 with valid_loss value: 1.1494574546813965.
Better model found at epoch 4 with valid_loss value: 0.6926746964454651.
Better model found at epoch 5 with valid_loss value: 0.601469874382019.
Better model found at epoch 9 with valid_loss value: 0.5294129848480225.
Better model found at epoch 10 with valid_loss value: 0.21726112067699432.
Better model found at epoch 11 with valid_loss value: 0.20636610686779022.
Better model found at epoch 13 with valid_loss value: 0.13778840005397797.




Fold 4


epoch,train_loss,valid_loss,accuracy,time
0,2.515908,3.622529,0.442857,01:13
1,1.122036,1.975238,0.638095,01:14
2,0.749274,2.727360,0.654762,01:14
3,0.564483,1.334962,0.607143,01:14
4,0.458534,0.762566,0.788095,01:13


Better model found at epoch 0 with valid_loss value: 3.622528553009033.
Better model found at epoch 1 with valid_loss value: 1.9752377271652222.
Better model found at epoch 3 with valid_loss value: 1.334962010383606.
Better model found at epoch 4 with valid_loss value: 0.7625659704208374.


epoch,train_loss,valid_loss,accuracy,time
0,2.515908,3.622529,0.442857,01:13
1,1.122036,1.975238,0.638095,01:14
2,0.749274,2.727360,0.654762,01:14
3,0.564483,1.334962,0.607143,01:14
4,0.458534,0.762566,0.788095,01:13
5,0.319431,0.544249,0.857143,01:13
6,0.287624,1.298222,0.628571,01:14
7,0.262916,0.441377,0.850000,01:13
8,0.235942,1.637814,0.695238,01:14
9,0.184500,0.150293,0.945238,01:13


Better model found at epoch 5 with valid_loss value: 0.5442490577697754.
Better model found at epoch 7 with valid_loss value: 0.44137710332870483.
Better model found at epoch 9 with valid_loss value: 0.15029308199882507.
Better model found at epoch 10 with valid_loss value: 0.1363690346479416.
Better model found at epoch 13 with valid_loss value: 0.12580570578575134.
Better model found at epoch 14 with valid_loss value: 0.10816697031259537.
Better model found at epoch 15 with valid_loss value: 0.1023009791970253.




Fold 7


KeyboardInterrupt: ignored

In [ ]:
ss[ss.columns[1:]] = np.mean([i.numpy() for i in predictions], axis=0)
ss.head()

,Word_id,hapana,kumi,mbili,moja,nane,ndio,nne,saba,sita,tano,tatu,tisa
0,id_jp2pxl0r84ya.wav,1.212510e-05,1.219315e-06,6.938043e-06,4.242263e-06,2.854734e-06,4.503219e-05,9.717359e-05,4.551821e-06,9.977739e-01,6.044260e-07,1.159074e-06,2.050244e-03
1,id_ndduqqvthbpx.wav,1.364954e-08,2.497533e-08,2.935750e-08,2.095295e-09,2.762485e-08,3.255530e-08,1.425581e-07,2.670823e-08,1.186409e-06,2.567384e-09,1.057834e-08,9.999986e-01
2,id_36oxymxfcm6q.wav,6.338138e-06,6.822980e-06,1.047654e-05,1.296558e-05,5.232308e-06,4.583628e-05,4.132303e-06,1.146476e-05,1.141568e-04,9.614543e-06,1.980021e-05,9.997532e-01
3,id_ue9b0to760pg.wav,6.908835e-06,3.197134e-06,9.751767e-08,3.932810e-08,7.983632e-06,1.271998e-07,9.707770e-08,4.287685e-07,2.960803e-08,9.999775e-01,3.489273e-06,9.402256e-08
4,id_prja4oprb914.wav,6.061909e-02,7.300982e-02,6.287435e-02,5.123470e-02,2.836692e-01,5.611872e-02,6.008373e-02,4.027027e-02,6.744351e-02,5.359995e-02,1.269318e-01,6.414486e-02


In [ ]:
from sklearn.metrics import log_loss 

val = pd.read_csv(path+'Validation.csv')[['Word_id', 'Swahili_word']]
chk = ss[ss.Word_id.apply(lambda x:x in val.Word_id.unique())]

log_loss(val.Swahili_word, chk[chk.columns[1:]])

0.07647987035133363

In [ ]:
name = "/content/sub1.csv"
ss.to_csv(name, index=False)